In [1]:
import warnings

warnings.filterwarnings("ignore")

import os
import re
import sys
from pathlib import Path

import numpy as np
import pandas as pd
from tqdm import tqdm

C:\Users\Utilisateur\miniconda3\envs\ds38\lib\site-packages\ipykernel\parentpoller.py:113: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  warnings.warn("""Parent poll failed.  If the frontend dies,


In [2]:
with open("d19_input.txt", "r") as f:
    lines = f.read().split("\n")
# lines = [(i[0], int(i[1:])) for i in lines]

In [3]:
lines[0], lines[-1]

('62: 93 93',
 'aaababbbabaababbabbbaabbbababaaaaaabbabbbbabbabaabababbbbbbababbabbbbaabbabababb')

In [4]:
rule_dict = {}
code_li = []
letter_dict = {-1: [""]}  # for single number

pattern = r"\d+"
for l in lines:
    if l == "":
        continue
    if l[0] in ["a", "b"]:
        code_li.append(l)
        continue
    nums = re.findall(pattern, l)
    nums = [int(n) for n in nums]
    if len(nums) == 3:
        if l.find("|") == -1:
            rule_dict[nums[0]] = [[nums[1], nums[2]], []]
        else:  # 93: 127 | 40
            rule_dict[nums[0]] = [[nums[1], -1], [nums[2], -1]]
            print(l)
    elif len(nums) == 5:
        rule_dict[nums[0]] = [[nums[1], nums[2]], [nums[3], nums[4]]]
    elif len(nums) == 2:  # 8: 42
        rule_dict[nums[0]] = [[nums[1], -1], []]
        print(l)
    elif len(nums) == 1:  # 127: "b"
        letter_dict[nums[0]] = [
            l[-2],
        ]
    else:
        print(f"error: {l}")

8: 42
93: 127 | 40


In [5]:
letter_dict

{-1: [''], 127: ['b'], 40: ['a']}

In [6]:
def get_comb(to_comb_li):
    if len(to_comb_li) == 2:
        left_num, right_num = to_comb_li[0], to_comb_li[1]
        if (left_num in letter_dict) and (right_num in letter_dict):
            return [
                l + r for l in letter_dict[left_num] for r in letter_dict[right_num]
            ]
    return to_comb_li

In [7]:
start_rule_num = 0

In [8]:
while start_rule_num not in letter_dict:
    # replace by letter_dict
    rule_dict = {k: [get_comb(v[0]), get_comb(v[1])] for k, v in rule_dict.items()}
    # put all letters dict to letter_dict
    for k, v in rule_dict.items():
        if all(isinstance(x, str) for x in v[0]) and all(
            isinstance(x, str) for x in v[1]
        ):
            letter_dict[k] = [*v[0], *v[1]]

In [9]:
len(letter_dict[start_rule_num])

2097152

In [10]:
sum(list(map(lambda s: 1 if s in letter_dict[start_rule_num] else 0, code_li)))

162

In [11]:
first_rule = [*rule_dict[42][0], *rule_dict[42][1]]  # all the combinations in 42
last_rule = [*rule_dict[31][0], *rule_dict[31][1]]  # all the combinations in 31

In [12]:
# all the combinations in 42 and 31 have 8 letters !!
np.unique(np.array([len(l) for l in first_rule])), np.unique(
    np.array([len(l) for l in last_rule])
)

(array([8]), array([8]))

In [13]:
# 0: 8,11
# 8: 42 | 42 8 --> so there will N times of "42" at the 1st part of string, N >=1
# 11: 42 31 | 42 11 31 --> so the second part of string has M times of "42" and M times of "31", M >=1
# So: the valid strings should contain X times of "42" and followed by Y times of "31", (X>=2)and(Y>=1)and(X>=Y+1)


def check_valid(s):
    if len(s) % 8 != 0:
        return 0
    s_li = [s[8 * i : 8 * i + 8] for i in range(len(s) // 8)] # separate
    for X, i in enumerate(s_li):
        if i not in first_rule:
            break
    if X < 2:
        return 0
    Y = 0
    for j in s_li[X:]:
        if j not in last_rule:
            return 0
        else:
            Y += 1
    if (Y==0) or (X < Y + 1):
        return 0
    return 1

In [14]:
sum(list(map(lambda ss: check_valid(ss), code_li)))

267